# Программирование на Python
## Семинар 4. Повторение основ
#### Задача с прошлого занятия
Мы хотим автоматизировать процесс перекладки текстовых файлов с данными из хранилища в БД (Postgres). Предположим, что таблицы каждый раз разные. Соответственно, перед записью данных необходимо создать таблицу. Это можно сделать примерно так:

```
CREATE TABLE distributors (
    did integer,
    name varchar(40)
);
```

PostgreSQL [располагает](https://www.postgresql.org/docs/current/datatype.html) следующими типами данных (это не все, что есть, но все, что нам сейчас пригодится):

<table>
    <tr>
        <th>Python</th>
        <th>PostgreSQL</th>
    </tr>
    <tr>
        <td>int</td>
        <td>smallint, integer, bigint</td>
    </tr>
    <tr>
        <td>float</td>
        <td>real</td>
    </tr>
    <tr>
        <td>bool</td>
        <td>boolean</td>
    </tr>
    <tr>
        <td>str</td>
        <td>varchar, text</td>
    </tr>
</table>

Реализуйте соответствующую функцию. Она должна иметь следующие аргументы:

- `filename` (строка) - путь к текстовому файлу;
- `sep` (строка, по дефолту для CSV) - разделитель;
- `tablename` (строка, по дефолту соответствует названию файла без расширения);
- `varnames` (либо должны быть предоставлены в виде списка, либо добываются из данных если `None`);
- `dtypes` (либо должны быть предоставлены в виде списка, либо добываются из данных если `None`).

Функция должна записывать получившийся скрипт в файл 'create_table.sql' и возвращать путь к этому файлу.

In [ ]:
# check if bool
# check if int
# check if float
# else str

In [58]:
big_list = list(range(100000))
big_set = set(range(100000))

In [59]:
%%timeit

_ = 50000 in big_list

705 µs ± 81.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [60]:
%%timeit

_ = 50000 in big_set

53.2 ns ± 1.67 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [61]:
_ = 1

In [63]:
a, b, _ = (4, False, 'a')

In [25]:
triplets = [(1, 2, 3), (3, 2, 1)]

for tpl in triplets:
    print(tpl)

(1, 2, 3)
(3, 2, 1)


In [ ]:
for tpl in triplets:
    val1, val2, val3 = tpl
    print(val1, val2, val3)

In [26]:
for val1, val2, val3 in triplets:
    print(val1, val2, val3)

1 2 3
3 2 1


In [10]:
all([True, True, 1, ''])

False

In [11]:
all(i == 2 for i in range(10))

False

In [44]:
import re

trash_symbols = '\t" \n'
pattern_int = re.compile('-{0,1}\d+')
pattern_float = re.compile('-{0,1}\d+\.\d*')
template = 'CREATE TABLE "{tablename}" (\n{pairs}\n);'

def check_bool(value: str) -> bool:
    return value.lower() in {'false', 'true'}

def check_int(value: str) -> bool:
    return pattern_int.fullmatch(value) is not None

def check_float(value: str) -> bool:
    return pattern_float.fullmatch(value) is not None

def check_dtype(vector, fun) -> bool:
    indicator = all(fun(value) for value in vector)

    return indicator

def guess_dtype(vector) -> str:
    if check_dtype(vector, check_bool):
        return 'boolean'
    elif check_dtype(vector, check_int):
        return 'integer'
    elif check_dtype(vector, check_float):
        return 'float'
    else:
        return 'text'

def extract_from_path(filename: str) -> str:
    slash_index = filename.rfind('/')  # path[::-1].find('/')
    dot_index = filename.rfind('.')
    
    return filename[slash_index + 1:dot_index]

def sql_from_file(
    filename: str,
    sep=',',
    tablename=None,
    varnames=None,
    dtypes=None,
    request_file='script.sql'
) -> str:
    '''
    filename: path to file with data
    sep: ...
    '''
    # load file
    with open(filename, 'r') as file:  # file = open(path, 'r')
        lines = []
    
        for line in file:  # iter(file) ...
            line_clean = [object.strip(trash_symbols) for object in line.split(sep)]
            
            lines.append(line_clean)

    # get tablename
    if tablename is None:
        tablename = extract_from_path(filename)

    # get varnames
    if varnames is None:
        varnames = lines[0].copy()  # list(lines[0])

    # reshape lines
    lines_t = zip(*lines[1:])
    
    # get dtypes
    if dtypes is None:
        dtypes = [guess_dtype(column) for column in lines_t]

    # create request
    pairs = ',\n'.join([f'    "{name}" {dtype}' for name, dtype in zip(varnames, dtypes)])
    request = template.format(tablename=tablename, pairs=pairs)
    
    # save request
    with open(request_file, 'w') as file:
        file.write(request)
        
    return request_file

In [45]:
path = '../Занятие 1/StudentsPerformance.csv'

script_file = sql_from_file(
    path
)

In [19]:
tname, vnames, types

('StudentsPerformance',
 ['gender',
  'race/ethnicity',
  'parental level of education',
  'lunch',
  'test preparation course',
  'math score',
  'reading score',
  'writing score'],
 ['text', 'text', 'text', 'text', 'text', 'integer', 'integer', 'integer'])

In [33]:
pairs = ',\n'.join([f'    "{name}" {dtype}' for name, dtype in zip(vnames, types)])
pairs

'    "gender" text,\n    "race/ethnicity" text,\n    "parental level of education" text,\n    "lunch" text,\n    "test preparation course" text,\n    "math score" integer,\n    "reading score" integer,\n    "writing score" integer'

In [40]:
template = 'CREATE TABLE "{tablename}" (\n{pairs}\n);'

In [41]:
print(template.format(tablename=tname, pairs=pairs))

CREATE TABLE "StudentsPerformance" (
    "gender" text,
    "race/ethnicity" text,
    "parental level of education" text,
    "lunch" text,
    "test preparation course" text,
    "math score" integer,
    "reading score" integer,
    "writing score" integer
);


In [ ]:
CREATE TABLE distributors (
    did integer,
    name varchar(40)
);

In [16]:
guess_dtype(check[-1])

'integer'

In [64]:
import re

In [75]:
148.

148.0

In [ ]:
'-{0,1}'

In [76]:
pattern_float = re.compile('-{0,1}\d+\.\d*')

In [67]:
pattern_int = re.compile('-{0,1}\d+')

In [74]:
pattern_int.fullmatch('-123.93')

In [68]:
pattern_int.search('-123')

<re.Match object; span=(0, 4), match='-123'>

In [69]:
pattern_int.search('-123amcskmcksa')

<re.Match object; span=(0, 4), match='-123'>

In [70]:
pattern_int.search('amcskmcksa-123')

<re.Match object; span=(10, 14), match='-123'>

In [71]:
pattern_int.search('-123amcskmcksa-123')

<re.Match object; span=(0, 4), match='-123'>

In [41]:
list_of_lists = [
    [1, 2, 3],
    [3, 2, 1]
]
lst = list_of_lists[1].copy()

In [42]:
lst.sort()

In [43]:
list_of_lists

[[1, 2, 3], [3, 2, 1]]

In [14]:
path = '../Занятие 1/StudentsPerformance.csv'

In [33]:
trash_symbols = '\t" \n'

with open(path, 'r') as file:  # file = open(path, 'r')
    lines = []

    for line in file:  # iter(file) ...
        line_clean = [object.strip(trash_symbols) for object in line.split(',')]
        
        lines.append(line_clean)

In [49]:
list(zip([1, 2, 3], ['a', 'b', 'c'], [True, False]))  # *args

[(1, 'a', True), (2, 'b', False)]

In [51]:
print([1, 2, 3], sep=':')

[1, 2, 3]


In [52]:
print(*[1, 2, 3], sep=':')

1:2:3


In [53]:
lines_t = list(zip(*lines[1:]))

In [56]:
lines[1]

['female',
 'group B',
 "bachelor's degree",
 'standard',
 'none',
 '72',
 '72',
 '74']

In [55]:
len(lines_t)

8

In [34]:
lines[0]

['gender',
 'race/ethnicity',
 'parental level of education',
 'lunch',
 'test preparation course',
 'math score',
 'reading score',
 'writing score']

#### Задача 1

Известно, что подготовка и исследование данных для машинного обучения занимают куда больше времени, чем собственно машинное обучение. В частности, существует такая процедура, как создание т. н. dummy-переменных.

![dummy](https://www.statology.org/wp-content/uploads/2021/02/dummyvartrap1-768x344.png)

В `pandas` уже есть функция, которая принимает на вход текстовую колонку таблицы и возвращает вместо нее много колонок с dummy-переменными. Однако проблема в том, что в ваших данных есть переменные, значения которых представляют из себя словосочетания со знаками препинания (количество таких слов формально не ограничено). Некоторые алгоритмы "любят" только простые названия переменных вида `variable` или `simple_variable`. Кроме того, так или иначе для последующей работы неплохо было бы стандартизировать все названия.

Поэтому вам нужно создать функцию, которая бы принимала на вход список из значений переменной и производила следующую предобработку всех значений в колонке:

- удаление всех знаков препинания;
- приведение всех букв к нижнему регистру;
- замену всех пробелов на нижнее подчеркивание;
- ограничение длины каждого слова первыми четырьмя буквами.

Функция должна возвращать словарь (т. н. mapper), по которому можно преобразовать переменную в новый вид (с помощью методов `.map()` / `.apply` - о них вам расскажут позднее). Ключами должны быть старые значения, а собственно значениями - новые.

Не всегда бывает так, что значения переменной написаны на латинице. Напишите вашу функцию таким образом, чтобы по дефолту она обрабатывала англоязычные переменные, но также имела возможность (при соответствующем значении аргумента) обработать и кириллические. Вам может пригодиться функция `translit` из модуля `transliterate` (пример работы с кириллицей можете найти ниже).

**Пример**

```
function input:
['Agree', 'Agree', 'Neither agree nor disagree', "Don't know", 'Neither agree nor disagree', 'Neither agree nor disagree', 'Disagree', 'No answer', 'Agree strongly', 'No answer', 'Agree', 'Refusal', 'Refusal', 'Disagree strongly', 'Disagree']

function return:
{
    'Agree strongly': 'agre_stro',
    'Agree': 'agre',
    'Neither agree nor disagree': 'neit_agre_nor_disa',
    'Disagree': 'disa',
    'Disagree strongly': 'disa_stro',
    'Refusal': 'refu',
    "Don't know": 'dont_know',
    'No answer': 'no_answ'
}
```

In [46]:
import re

from string import punctuation

In [48]:
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
+7(000)985-

In [53]:
pattern = re.compile('[-\(\)]')

In [54]:
pattern.sub('', '+7(000)985-00-00')

'+70009850000'

In [ ]:
# пример работы функции translit

from transliterate import translit

print(translit('Полностью не согласен', 'ru', reversed=True))

In [ ]:
# наш код здесь

#### Задача 2
Предположим, что перед нами стоит задача создать dummy-переменные в PostgreSQL. К сожалению, это не `pandas`, и готовой функции на этот случай не предусмотрено. Однако вы знаете, что можно создавать новые переменные базируясь на значении старых используя конструкцию case...when...then...end:

```
PostgreSQL:
CASE WHEN <condition> THEN <value1> ELSE <value2> END AS <variable_name>
В ТОМ СЛУЧАЕ КОГДА <условие> ТОГДА <значение1> ИНАЧЕ <значение2> КОНЕЦ НАЗВАТЬ <имя_переменной>

Python:
if <condition>:
    <value1>
else:
    <value2>
```

Если категорий немного, то написать такой код несложно. Однако что делать, если их, к примеру, 100? В этом случае придется писать `CASE WHEN` столько раз, сколько уникальных значений содержит ваша переменная, да еще и придумывать каждый раз соответствующее название.

Используйте свои знания Python, чтобы автоматизировать процесс написания SQL-запроса. Напишите соответствующую функцию. Для автоматизированной генерации названия переменной используйте наработки предыдущей задачи. Учтите, что значения переменной не обязательно будут написаны на латинице.

На выходе функция должна печатать (в таком же формате, как в примере ниже) законченный блок SQL-запроса.

**Пример**
```
function return:
    CASE WHEN gincdif = 'Agree strongly' THEN 1 ELSE 0 END AS agre_stro,
    CASE WHEN gincdif = 'Agree' THEN 1 ELSE 0 END AS agre,
    CASE WHEN gincdif = 'Neither agree nor disagree' THEN 1 ELSE 0 END AS neit_agre_nor_disa,
    CASE WHEN gincdif = 'Disagree' THEN 1 ELSE 0 END AS disa,
    CASE WHEN gincdif = 'Disagree strongly' THEN 1 ELSE 0 END AS disa_stro,
    CASE WHEN gincdif = 'Refusal' THEN 1 ELSE 0 END AS refu,
    CASE WHEN ginsdif = "Don't know" THEN 1 ELSE 0 END AS dont_know,
    CASE WHEN ginsdif = "No answer" THEN 1 ELSE 0 END AS no_answ
```

In [ ]:
# наш код здесь

#### Задача к занятию

In [ ]:
import os
def get_tree(folder_path, level=0):
    dirnames = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    filenames = [d for d in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, d))]
    
    sorted_list = sorted(dirnames + filenames)
    
    for s in sorted_list:
        if s in filenames:
            print("   " * level, s)

        else:
            print("   " * level, s)
            get_tree(os.path.join(folder_path, s), level=level+1)

In [1]:
from pathlib import Path

In [2]:
current_path = Path('.')

In [3]:
current_path.resolve()

WindowsPath('C:/Users/M050/Desktop/Работа/SkillFactory/Науки о данных/Занятие 4')

In [5]:
current_path.resolve().parent

WindowsPath('C:/Users/M050/Desktop/Работа/SkillFactory/Науки о данных')

In [7]:
current_path.resolve() / 'folder'

WindowsPath('C:/Users/M050/Desktop/Работа/SkillFactory/Науки о данных/Занятие 4/folder')

#### Задача на дом
На вход подаются значения через ':' (ОДНА СТРОКА):

```
value1:value2:value3:value4
value1:value2:value3
value1:value2:value3:
:value1:value2:value3

:::
```

Необходимо создать словарь, в котором ключами будут значения с нечетной позицией, а значениями - с четной. Необходимо, чтобы из этого словаря можно было восстановить входную строку (но пары могут меняться местами).

```
In: value1:value2:value3:value4
Out: value3:value4:value1:value2
```